In [14]:
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os

import shutil
import sklearn
from sklearn.model_selection import KFold
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score, roc_curve,precision_recall_curve, auc

import sys
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_GSCV import * # RF_GSCV contains the calculate metrics function to get the TP, TN, FP, FN scores 
from RF_atomver import prediction_type

In [15]:

class DirichletGPModel(ExactGP):
    """
    A Dirichlet Gaussian Process (GP) model for multi-class classification.
    This model uses a Gaussian Process with a Dirichlet prior to handle multi-class classification tasks.
    It extends the ExactGP class from GPyTorch, a library for Gaussian Processes in PyTorch.
    Attributes:
        mean_module (gpytorch.means.ConstantMean): The mean module for the GP, initialized with a constant mean function for each class.
        covar_module (gpytorch.kernels.ScaleKernel): The covariance module for the GP, using a scaled RBF kernel for each class.

    Args:
        train_x (torch.Tensor): Training data features.
        train_y (torch.Tensor): Training data labels.
        likelihood (gpytorch.likelihoods.Likelihood): The likelihood function.
        num_classes (int): The number of classes for the classification task.
    """
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = ScaleKernel(MaternKernel(nu=0.5, batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,))
        )

    def forward(self, x):
        """
        Forward pass through the GP model.
        Args:
            x (torch.Tensor): Input data features.
        Returns:
            gpytorch.distributions.MultivariateNormal: The multivariate normal distribution representing the GP posterior.
        """
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [16]:
class Trainer: 
    def __init__(self,model, likelihood, iterations): 
        self.model = model
        self.likelihood = likelihood 
        smoke_test = ('CI' in os.environ)
        self.n_iterations = 2 if smoke_test else iterations
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        self.loss_fn = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)
        
    def train(self, train_x, train_y): 
        self.model.train()
        self.likelihood.train()
        predictions = [] 
        for i in range(self.n_iterations): 
            self.optimizer.zero_grad()
            output = self.model(train_x)
            loss = -self.loss_fn(output, self.likelihood.transformed_targets).sum()
            loss.backward()
            if (i%10==0): 
                print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                    i + 1, self.n_iterations, loss.item(),
                    self.model.covar_module.base_kernel.lengthscale.mean().item(),
                    self.model.likelihood.second_noise_covar.noise.mean().item()
                ))
             
            self.optimizer.step() 
    def predict(self, input): 
        """
        Make predictions using the GP model.

        Args:
            input (torch.Tensor): The input data for making predictions.
        
        Returns:
            dist (gpytorch.distributions.MultivariateNormal): The distribution representing the GP posterior.
            observed_pred (gpytorch.distributions.MultivariateNormal): The predicted distribution considering the likelihood.
            pred_means (torch.Tensor): The means of the predicted distributions.
            class_pred (torch.Tensor): The predicted class labels.
        """
        self.model.eval()
        self.likelihood.eval()

        with gpytorch.settings.fast_pred_var(), torch.no_grad():
            dist = self.model(input)     # output distribution
            pred_means = dist.loc          # means of distributino 
            observed_pred = self.likelihood(self.model(input))    # likelihood predictions mean and var  

            class_pred = self.model(input).loc.max(0)[1]
            
        return dist, observed_pred, pred_means, class_pred
    

    def evaluate(self, x_input, y_true): 
        """
        Evaluate the GP model.

        Args:
            x_input (torch.Tensor): The input data features.
            y_true (torch.Tensor): The true labels for the input data.
        
        Returns:
            y_pred (numpy.ndarray): The predicted class labels.
        """
        y_pred = self.model(x_input).loc.max(0)[1].numpy()
        
        return y_pred

    def gp_results(self, x_input, y_true, plot_title=None): 
        """
        Calculate evaluation metrics and print results.

        Args:
            x_input (torch.Tensor): The input data features.
            y_true (torch.Tensor or numpy.ndarray): The true labels for the input data.
            plot_title (str, optional): The title for the confusion matrix plot.
        
        Returns:
            dict: A dictionary containing evaluation metrics and confusion matrix components.
        """
        y_pred = self.evaluate(x_input, y_true) 
        if isinstance(y_true, torch.Tensor):
            y_true = y_true.numpy().reshape(-1)
        # plot_confusion_matrix(y_true, y_pred, ['0','1'], title=plot_title)
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        dist = self.model(x_input)     # get predicted distributions 
        pred_means = dist.loc          # means for predicted dist  

        recall = recall_score(y_true, y_pred)
        tp, tn, fp, fn = calculate_metrics(y_true, y_pred) 
        sensitivity = tp / (tp + fn) 
        specificity = tn / (tn + fp) 
        cm = confusion_matrix(y_true, y_pred)
        print(f'accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, specificity: {specificity:.4f}, cm: {cm}')
        return {'accuracy': accuracy, 'precision': precision,  'recall':recall, 'specificity':specificity, 'TN': tn, 'FN': fn, 'FP': fp, 'TP': tp }

       

In [17]:
def make_torch_tens_float(filepath, filename): 
    trainX_df = pd.read_csv(filepath+filename+'_trainX.csv')
    trainy_df = pd.read_csv(filepath+filename+'_train_y.csv')
    testX_df = pd.read_csv(filepath+filename+'_testX.csv')
    testy_df = pd.read_csv(filepath+filename+'_test_y.csv')

    train_x_temp = trainX_df.to_numpy().astype("double") # double 
    test_x_temp = testX_df.to_numpy().astype("double") #double 
    
    train_y_temp = trainy_df.to_numpy().flatten().astype("double") #double 
    test_y_temp = testy_df.to_numpy().flatten().astype("double") #double 
    
    # trainX = torch.from_numpy(train_x_temp)
    # trainy = torch.from_numpy(train_y_temp)
    # testX = torch.from_numpy(test_x_temp)
    # testy = torch.from_numpy(test_y_temp)
    trainX = torch.as_tensor(train_x_temp, dtype=torch.float32)
    trainy = torch.as_tensor(train_y_temp, dtype=torch.float32)
    testX = torch.as_tensor(test_x_temp, dtype=torch.float32)
    testy = torch.as_tensor(test_y_temp, dtype=torch.float32)
    return trainX, trainy, testX, testy

In [18]:
GP_path= '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/'
def save_results(trainX, trainy, testX, testy, root_name, n_iterations=300, n_samples=100):
    """
    Train a Dirichlet Gaussian Process model and save the training and test performance results.

    This function trains a Dirichlet GP model on the given training data, evaluates it on both the training
    and test data, and saves various performance metrics and predictions to pandas DataFrames.

    Args:
        trainX (torch.Tensor): The training data features.
        trainy (torch.Tensor): The training data labels.
        testX (torch.Tensor): The test data features.
        testy (torch.Tensor): The test data labels.
        root_name (str): The root name used for labeling the model in the results.
        n_iterations (int, optional): The number of training iterations. Default is 300.
        n_samples (int, optional): The number of samples for prediction. Default is 100.

    Returns:
        train_perf_df (pd.DataFrame): DataFrame containing performance metrics and predictions for the training data.
        test_perf_df (pd.DataFrame): DataFrame containing performance metrics and predictions for the test data.
    """
    likelihood = DirichletClassificationLikelihood(trainy.long(), learn_additional_noise=True)
    model = DirichletGPModel(trainX, likelihood.transformed_targets, likelihood, num_classes=likelihood.num_classes)
    # n_iterations = 300
    trainer = Trainer(model, likelihood, n_iterations)
    trainer.train(trainX, trainy) 
  
    train_dist, train_observed_pred, train_pred_means, train_pred  = trainer.predict(trainX)
    train_results = trainer.gp_results(trainX, trainy)
    test_dist, test_observed_pred, test_pred_means, test_pred  = trainer.predict(testX)
    test_results = trainer.gp_results(testX, testy)
    
    train_observed_pred.mean.numpy()
    train_pred_variance2D = train_observed_pred.variance.numpy()
    test_observed_pred.mean.numpy()
    test_pred_variance2D=test_observed_pred.variance.numpy()
    
    train_pred_samples = train_dist.sample(torch.Size((256,))).exp()
    train_probabilities = (train_pred_samples / train_pred_samples.sum(-2, keepdim=True)).mean(0)

    train_prob_stds = (train_pred_samples / train_pred_samples.sum(-2, keepdim=True)).std(0)

    test_pred_samples = test_dist.sample(torch.Size((100,))).exp()

    test_probabilities = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).mean(0)
    test_prob_stds = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).std(0)

 
    train_perf_df = pd.DataFrame()
    test_perf_df = pd.DataFrame()
    train_perf_df['mean_pred_class0'] = train_observed_pred.mean.numpy()[0,]
    train_perf_df['mean_pred_class1'] = train_observed_pred.mean.numpy()[1,]
    train_perf_df['y'] = trainy
    train_perf_df['y_pred'] = train_pred_means.max(0)[1]
    train_perf_df['var_pred_class0']=train_observed_pred.variance.numpy()[0,]
    train_perf_df['var_pred_class1']=train_observed_pred.variance.numpy()[1,]
    train_perf_df['pred_prob_class0'] = train_probabilities.numpy()[0,]
    train_perf_df['pred_prob_class1'] = train_probabilities.numpy()[1,]
    train_perf_df['pred_prob_std_class0'] = train_prob_stds.numpy()[0,]
    train_perf_df['pred_prob_std_class1'] = train_prob_stds.numpy()[1,]
    train_perf_df['model'] = f'{root_name}_GP_Dirichlet_matern'
    train_perf_df['subset'] = 'train' 
    train_cm = confusion_matrix(trainy, train_perf_df['y_pred'])
    cm_flattened = train_cm.flatten().tolist()
    train_perf_df['cm']= [cm_flattened]* len(train_perf_df)
    train_perf_df['prediction_type'] = train_perf_df.apply(lambda x: prediction_type(x['y'], x['y_pred']), axis=1)
    train_perf_df['ROC-AUC'] = roc_auc_score(trainy, train_perf_df['y_pred'])
    train_perf_df['MCC'] = matthews_corrcoef(trainy, train_perf_df['y_pred'])
    train_perf_df['Balanced Accuracy'] = balanced_accuracy_score(trainy, train_perf_df['y_pred'])
    train_perf_df['f1'] = f1_score(trainy, train_perf_df['y_pred'])
    
    test_perf_df['mean_pred_class0'] = test_observed_pred.mean.numpy()[0,]
    test_perf_df['mean_pred_class1'] = test_observed_pred.mean.numpy()[1,]
    test_perf_df['y'] = testy
    test_perf_df['y_pred'] = test_pred_means.max(0)[1]
    test_perf_df['var_pred_class0']=test_observed_pred.variance.numpy()[0,]
    test_perf_df['var_pred_class1']=test_observed_pred.variance.numpy()[1,]
    test_perf_df['pred_prob_class0'] = test_probabilities.numpy()[0,]
    test_perf_df['pred_prob_class1'] = test_probabilities.numpy()[1,]
    test_perf_df['pred_prob_std_class0'] =test_prob_stds.numpy()[0,]
    test_perf_df['pred_prob_std_class1'] = test_prob_stds.numpy()[1,]
    test_perf_df['model'] = f'{root_name}_GP_Dirichlet_matern'
    test_perf_df['subset'] = 'test' 
    test_cm = confusion_matrix(testy, test_perf_df['y_pred'])
    test_cm_flattened = test_cm.flatten().tolist()
    test_perf_df['cm']= [test_cm_flattened]* len(test_perf_df)
    test_perf_df['prediction_type'] = test_perf_df.apply(lambda x: prediction_type(x['y'], x['y_pred']), axis=1)
    test_perf_df['ROC-AUC'] = roc_auc_score(testy, test_perf_df['y_pred'])
    test_perf_df['MCC'] = matthews_corrcoef(testy, test_perf_df['y_pred'])
    test_perf_df['Balanced Accuracy'] = balanced_accuracy_score(testy, test_perf_df['y_pred'])
    test_perf_df['f1'] = f1_score(testy, test_perf_df['y_pred'])
    with open(f'{GP_path}{root_name}_GP_Dirichlet_matern_model.pkl', 'wb') as f: 
        pickle.dump(model,f)
    with open(f'{GP_path}{root_name}_GP_Dirichlet_matern_likelihood.pkl', 'wb') as f: 
        pickle.dump(likelihood,f)
    for k, val in train_results.items(): 
        train_perf_df[k] = val
    for k, val in test_results.items():
        test_perf_df[k] = val
    return train_perf_df, test_perf_df


In [19]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
feat_types = ['moe', 'mfp']
neks = ['2', '3', '5', '9']
GP_path= '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/' 
for nek in neks:
    print(f'NEK{nek}')
    bind_inhib = ['binding', 'inhibition']
    if nek in ['3','5']: 
        bind_inhib = ['binding']
    else: 
        bind_inhib = ['binding', 'inhibition']
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind' 
        if bi == 'inhibition': 
            this_bi = 'inhib'
        nek_path= f'{data_dir}NEK{nek}/{this_bi}/'
        for feat in feat_types: 
            for samp in samplings: 
                print(f'NEK{nek} {bi} {feat} {samp}')
                file_root = f'NEK{nek}_{bi}_{feat}_{samp}'
                trainX, trainy, testX, testy = make_torch_tens_float(nek_path,file_root) 
                print(f'trainX:{trainX.shape}, train y: {trainy.shape}, testX: {testX.shape}, test y: {testy.shape}')
                train_perf_df, test_perf_df = save_results(trainX, trainy, testX, testy,file_root, n_iterations=300)
                train_perf_df['NEK'] = f'NEK{nek}_{bi}'
                train_perf_df['strategy'] = f'{samp}'
                train_perf_df['feat_type'] = f'{feat}'
                test_perf_df['NEK'] = f'NEK{nek}_{bi}'
                test_perf_df['strategy'] = f'{samp}'
                test_perf_df['feat_type'] = f'{feat}'
                train_perf_df.to_csv(f'{GP_path}{file_root}_train_GP_matern.csv',index=False) 
                test_perf_df.to_csv(f'{GP_path}{file_root}_test_GP_matern.csv',index=False) 
                print() 

NEK2
NEK2 binding moe scaled
trainX:torch.Size([1125, 306]), train y: torch.Size([1125]), testX: torch.Size([283, 306]), test y: torch.Size([283])
Iter 1/300 - Loss: 7.108   lengthscale: 0.693   noise: 0.693
Iter 11/300 - Loss: 5.483   lengthscale: 1.119   noise: 0.809
Iter 21/300 - Loss: 4.469   lengthscale: 1.801   noise: 1.072
Iter 31/300 - Loss: 3.758   lengthscale: 2.796   noise: 1.316
Iter 41/300 - Loss: 3.394   lengthscale: 4.012   noise: 1.427
Iter 51/300 - Loss: 3.283   lengthscale: 5.080   noise: 1.419
Iter 61/300 - Loss: 3.238   lengthscale: 5.995   noise: 1.345
Iter 71/300 - Loss: 3.212   lengthscale: 6.756   noise: 1.240
Iter 81/300 - Loss: 3.197   lengthscale: 7.395   noise: 1.121
Iter 91/300 - Loss: 3.169   lengthscale: 7.970   noise: 0.998
Iter 101/300 - Loss: 3.152   lengthscale: 8.453   noise: 0.876
Iter 111/300 - Loss: 3.145   lengthscale: 8.898   noise: 0.761
Iter 121/300 - Loss: 3.131   lengthscale: 9.313   noise: 0.659
Iter 131/300 - Loss: 3.116   lengthscale: 9.7

/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


accuracy: 0.9600, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1080    0]
 [  45    0]]
accuracy: 0.9576, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[271   0]
 [ 12   0]]


/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



NEK2 binding moe UNDER
trainX:torch.Size([90, 306]), train y: torch.Size([90]), testX: torch.Size([283, 306]), test y: torch.Size([283])
Iter 1/300 - Loss: 7.107   lengthscale: 0.693   noise: 0.693
Iter 11/300 - Loss: 5.931   lengthscale: 1.264   noise: 1.297
Iter 21/300 - Loss: 5.437   lengthscale: 2.051   noise: 1.982
Iter 31/300 - Loss: 5.260   lengthscale: 3.155   noise: 2.571
Iter 41/300 - Loss: 5.202   lengthscale: 4.419   noise: 3.001
Iter 51/300 - Loss: 5.183   lengthscale: 5.572   noise: 3.285
Iter 61/300 - Loss: 5.175   lengthscale: 6.518   noise: 3.455
Iter 71/300 - Loss: 5.170   lengthscale: 7.291   noise: 3.539
Iter 81/300 - Loss: 5.167   lengthscale: 7.937   noise: 3.563
Iter 91/300 - Loss: 5.164   lengthscale: 8.492   noise: 3.546
Iter 101/300 - Loss: 5.162   lengthscale: 8.981   noise: 3.504
Iter 111/300 - Loss: 5.160   lengthscale: 9.417   noise: 3.444
Iter 121/300 - Loss: 5.158   lengthscale: 9.811   noise: 3.372
Iter 131/300 - Loss: 5.157   lengthscale: 10.167   noi

...

Iter 1/300 - Loss: 6.409   lengthscale: 0.693   noise: 0.693
Iter 11/300 - Loss: 4.962   lengthscale: 1.307   noise: 0.844
Iter 21/300 - Loss: 4.193   lengthscale: 2.093   noise: 0.835
Iter 31/300 - Loss: 3.863   lengthscale: 2.836   noise: 0.526
Iter 41/300 - Loss: 3.705   lengthscale: 3.373   noise: 0.245
Iter 51/300 - Loss: 3.618   lengthscale: 3.748   noise: 0.107
Iter 61/300 - Loss: 3.574   lengthscale: 4.032   noise: 0.055
Iter 71/300 - Loss: 3.549   lengthscale: 4.267   noise: 0.035
Iter 81/300 - Loss: 3.532   lengthscale: 4.474   noise: 0.025
Iter 91/300 - Loss: 3.519   lengthscale: 4.664   noise: 0.020
Iter 101/300 - Loss: 3.509   lengthscale: 4.841   noise: 0.017
Iter 111/300 - Loss: 3.502   lengthscale: 5.007   noise: 0.014
Iter 121/300 - Loss: 3.495   lengthscale: 5.165   noise: 0.013
Iter 131/300 - Loss: 3.490   lengthscale: 5.316   noise: 0.011
Iter 141/300 - Loss: 3.485   lengthscale: 5.461   noise: 0.010
Iter 151/300 - Loss: 3.480   lengthscale: 5.600   noise: 0.009
Ite

/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 

accuracy: 0.9982, precision: 0.9964, recall: 1.0000, specificity: 0.9964, cm: [[279   1]
 [  0 280]]
accuracy: 0.9375, precision: 0.8333, recall: 0.5556, specificity: 0.9859, cm: [[70  1]
 [ 4  5]]

NEK9 inhibition mfp ADASYN
trainX:torch.Size([555, 2048]), train y: torch.Size([555]), testX: torch.Size([80, 2048]), test y: torch.Size([80])
Iter 1/300 - Loss: 6.238   lengthscale: 0.693   noise: 0.693
Iter 11/300 - Loss: 4.909   lengthscale: 1.307   noise: 0.794
Iter 21/300 - Loss: 4.190   lengthscale: 2.092   noise: 0.797
Iter 31/300 - Loss: 3.857   lengthscale: 2.844   noise: 0.490
Iter 41/300 - Loss: 3.694   lengthscale: 3.392   noise: 0.222
Iter 51/300 - Loss: 3.610   lengthscale: 3.772   noise: 0.096
Iter 61/300 - Loss: 3.570   lengthscale: 4.056   noise: 0.050
Iter 71/300 - Loss: 3.549   lengthscale: 4.287   noise: 0.032
Iter 81/300 - Loss: 3.534   lengthscale: 4.487   noise: 0.024
Iter 91/300 - Loss: 3.525   lengthscale: 4.669   noise: 0.019
Iter 101/300 - Loss: 3.515   lengthscal

/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 

accuracy: 1.0000, precision: 1.0000, recall: 1.0000, specificity: 1.0000, cm: [[280   0]
 [  0 275]]
accuracy: 0.9375, precision: 0.8333, recall: 0.5556, specificity: 0.9859, cm: [[70  1]
 [ 4  5]]



In [20]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
GP_path= '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/' 
train_results = [] 
test_results = [] 
for nek in neks:
    bind_inhib = ['binding', 'inhibition']
    if nek in ['3','5']: 
        bind_inhib = ['binding']
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind' 
        if bi == 'inhibition': 
            this_bi = 'inhib'
        nek_path= f'{data_dir}NEK{nek}/{this_bi}/'
        for feat in feat_types: 
            for samp in samplings: 
                print(f'NEK{nek} {this_bi} {feat} {samp}')
                file_root = f'NEK{nek}_{bi}_{feat}_{samp}'
                train = pd.read_csv(f'{GP_path}{file_root}_train_GP_matern.csv').iloc[0]
                test = pd.read_csv(f'{GP_path}{file_root}_test_GP_matern.csv').iloc[0]
                train_results.append(train)
                test_results.append(test) 

NEK2 bind moe scaled
NEK2 bind moe UNDER
NEK2 bind moe SMOTE
NEK2 bind moe ADASYN
NEK2 bind mfp scaled
NEK2 bind mfp UNDER
NEK2 bind mfp SMOTE
NEK2 bind mfp ADASYN
NEK2 inhib moe scaled
NEK2 inhib moe UNDER
NEK2 inhib moe SMOTE
NEK2 inhib moe ADASYN
NEK2 inhib mfp scaled
NEK2 inhib mfp UNDER
NEK2 inhib mfp SMOTE
NEK2 inhib mfp ADASYN
NEK3 bind moe scaled
NEK3 bind moe UNDER
NEK3 bind moe SMOTE
NEK3 bind moe ADASYN
NEK3 bind mfp scaled
NEK3 bind mfp UNDER
NEK3 bind mfp SMOTE
NEK3 bind mfp ADASYN
NEK5 bind moe scaled
NEK5 bind moe UNDER
NEK5 bind moe SMOTE
NEK5 bind moe ADASYN
NEK5 bind mfp scaled
NEK5 bind mfp UNDER
NEK5 bind mfp SMOTE
NEK5 bind mfp ADASYN
NEK9 bind moe scaled
NEK9 bind moe UNDER
NEK9 bind moe SMOTE
NEK9 bind moe ADASYN
NEK9 bind mfp scaled
NEK9 bind mfp UNDER
NEK9 bind mfp SMOTE
NEK9 bind mfp ADASYN
NEK9 inhib moe scaled
NEK9 inhib moe UNDER
NEK9 inhib moe SMOTE
NEK9 inhib moe ADASYN
NEK9 inhib mfp scaled
NEK9 inhib mfp UNDER
NEK9 inhib mfp SMOTE
NEK9 inhib mfp ADASYN


In [21]:
metric_cols = ['model','NEK', 'strategy', 'feat_type','cm','prediction_type', 'recall','ROC-AUC', 'MCC',
       'Balanced Accuracy', 'f1', 'accuracy', 'precision',
       'specificity', 'TN', 'FN', 'FP','TP']

train_df =  pd.DataFrame(train_results,columns=metric_cols)

train_df.to_csv(f'{GP_path}GP_maternkern_train_results.csv', index=False)
test_df =  pd.DataFrame(test_results,columns=metric_cols)
test_df.to_csv(f'{GP_path}GP_maternkern_test_results.csv',index=False)


In [22]:
test_df = pd.read_csv(GP_path+'GP_maternkern_test_results.csv')
test_df

,model,NEK,strategy,feat_type,cm,prediction_type,recall,ROC-AUC,MCC,Balanced Accuracy,f1,accuracy,precision,specificity,TN,FN,FP,TP
0,NEK2_binding_moe_scaled_GP_Dirichlet_matern,NEK2_binding,scaled,moe,"[271, 0, 12, 0]",TN,0.000000,0.500000,0.000000,0.500000,0.000000,0.957597,0.000000,1.000000,271,12,0,0
1,NEK2_binding_moe_UNDER_GP_Dirichlet_matern,NEK2_binding,UNDER,moe,"[212, 59, 5, 7]",TN,0.583333,0.682811,0.174223,0.682811,0.179487,0.773852,0.106061,0.782288,212,5,59,7
2,NEK2_binding_moe_SMOTE_GP_Dirichlet_matern,NEK2_binding,SMOTE,moe,"[270, 1, 9, 3]",TN,0.250000,0.623155,0.420460,0.623155,0.375000,0.964664,0.750000,0.996310,270,9,1,3
3,NEK2_binding_moe_ADASYN_GP_Dirichlet_matern,NEK2_binding,ADASYN,moe,"[271, 0, 9, 3]",TN,0.250000,0.625000,0.491899,0.625000,0.400000,0.968198,1.000000,1.000000,271,9,0,3
4,NEK2_binding_mfp_scaled_GP_Dirichlet_matern,NEK2_binding,scaled,mfp,"[271, 0, 12, 0]",TN,0.000000,0.500000,0.000000,0.500000,0.000000,0.957597,0.000000,1.000000,271,12,0,0
5,NEK2_binding_mfp_UNDER_GP_Dirichlet_matern,NEK2_binding,UNDER,mfp,"[4, 267, 0, 12]",FP,1.000000,0.507380,0.025196,0.507380,0.082474,0.056537,0.043011,0.014760,4,0,267,12
6,NEK2_binding_mfp_SMOTE_GP_Dirichlet_matern,NEK2_binding,SMOTE,mfp,"[271, 0, 10, 2]",TN,0.166667,0.583333,0.400918,0.583333,0.285714,0.964664,1.000000,1.000000,271,10,0,2
7,NEK2_binding_mfp_ADASYN_GP_Dirichlet_matern,NEK2_binding,ADASYN,mfp,"[271, 0, 10, 2]",TN,0.166667,0.583333,0.400918,0.583333,0.285714,0.964664,1.000000,1.000000,271,10,0,2
8,NEK2_inhibition_moe_scaled_GP_Dirichlet_matern,NEK2_inhibition,scaled,moe,"[381, 0, 28, 0]",TN,0.000000,0.500000,0.000000,0.500000,0.000000,0.931540,0.000000,1.000000,381,28,0,0
9,NEK2_inhibition_moe_UNDER_GP_Dirichlet_matern,NEK2_inhibition,UNDER,moe,"[331, 50, 6, 22]",TN,0.785714,0.827240,0.433967,0.827240,0.440000,0.863081,0.305556,0.868766,331,6,50,22
